In [1]:
import pandas as pd

data = pd.read_csv("assignment1/dataset_mood_smartphone.csv")
print(data)

        Unnamed: 0       id                     time        variable   value
0                1  AS14.01  2014-02-26 13:00:00.000            mood   6.000
1                2  AS14.01  2014-02-26 15:00:00.000            mood   6.000
2                3  AS14.01  2014-02-26 18:00:00.000            mood   6.000
3                4  AS14.01  2014-02-26 21:00:00.000            mood   7.000
4                5  AS14.01  2014-02-27 09:00:00.000            mood   6.000
...            ...      ...                      ...             ...     ...
376907     2770399  AS14.30  2014-04-11 07:51:16.948  appCat.weather   8.032
376908     2772465  AS14.30  2014-04-19 11:00:32.747  appCat.weather   3.008
376909     2774026  AS14.30  2014-04-26 10:19:07.434  appCat.weather   7.026
376910     2774133  AS14.30  2014-04-27 00:44:48.450  appCat.weather  23.033
376911     2784435  AS14.32  2014-04-07 18:25:14.036  appCat.weather  22.431

[376912 rows x 5 columns]


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from pandas import DataFrame

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from pandas import DataFrame

user_temp = "AS14.02"

user = data[data['id'] == user_temp]
user_mood = user[user["variable"] == "mood"]

avgs = []

times = list(user_mood["time"])
moods = list(user_mood["value"])


size = int(len(moods) * 0.8)
train, test = moods[0:size], moods[size:len(moods)]
time_test = times[size:len(times)]
history = [x for x in train]
predictions = list()

date_avg_pred = {}
accum_pred = 0
temp_counter = 0
counter = 0

df_data =  []

for t in range(len(test)):
    model = ARIMA(history, order=(1, 1, 1))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    temp_t = time_test[t]
    history.append(obs)
    
    if int(temp_t[8:10]) != counter and t != 0:
        average = accum_pred / temp_counter
        print("Day = {0}, Accumulated = {1}, Prediction in day = {2}, Average = {3}".format(time_test[t-1][0:10], accum_pred, temp_counter, average))
        df_data.append([user_temp, time_test[t-1][0:10], average])
        date_avg_pred[counter] = average
        accum_pred = 0
        temp_counter = 0
        
#     print("{}".format(temp_t[8:10]))
#     print('%f' % (yhat))
        
    accum_pred += yhat[0]
    
    if t == len(test) - 1:
        average = accum_pred / (temp_counter + 1)
        print("Day = {0}, Accumulated = {1}, Prediction in day = {2}, Average = {3}".format(time_test[t][0:10], accum_pred, temp_counter + 1, average))
        df_data.append([user_temp, time_test[t][0:10], average])
        date_avg_pred[counter] = average
        
    counter = int(temp_t[8:10])
    temp_counter += 1
    
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
df_x = pd.DataFrame(df_data, columns=["id", "date", "moodPred"])
print(df_x)
# plot
# plt.plot(test)
# plt.plot(predictions, color='red')
# plt.show()

In [ ]:
df_x

In [ ]:
date_avg_pred

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

user_dict = []

for user in data.id.unique():
    user_temp = user
    user = data[data['id'] == user]
    user_mood = user[user["variable"] == "mood"]

    avgs = []

    moods = list(user_mood["value"])

    size = int(len(moods) * 0.80)
    train, test = moods[0:size], moods[size:len(moods)]
    
    for q in [0, 1, 2, 3]:
        for p in [0, 1, 2, 3, 4, 5]:
            print(user_temp, p, 0, q)
            history = [x for x in train]
            predictions = list()
            try:
                for t in range(len(test)):
                    
                    model = ARIMA(history, order=(p, 0, q))
                    model_fit = model.fit(disp=0)
                    output = model_fit.forecast()
                    yhat = output[0]
                    predictions.append(yhat)
                    obs = test[t]
                    history.append(obs)
                    print('predicted=%f, expected=%f' % (yhat, obs))
                error = mean_squared_error(test, predictions)
                print('Test MSE: %.3f' % error)
                user_dict.append([user_temp, (p, 0, q), error])
            except:
                print("Didn't work for {0}, {1}, {2}".format(p, 0, q))
                pass
    # plot
#     plt.plot(test)
#     plt.plot(predictions, color='red')
#     plt.show()

In [ ]:
users = list(data.id.unique())

In [ ]:
print(users)

In [ ]:
# print(user_dict[0][0]["id"][0])

user_values = {}

i = 0
for instance in user_dict:
    if list(instance[0]["id"])[0] not in user_values:
        user_values[list(instance[0]["id"])[0]] = []
    user_values[list(instance[0]["id"])[0]].append([instance[1], instance[2]])

In [ ]:
for user_value in user_values:
    temp = np.array(user_values[user_value])
    index = np.where(temp[:,1] == np.amin(temp[:,1]))
    print(user_value, user_values[user_value][index[0][0]])

In [ ]:
import csv
with open('dict.csv', 'w', newline="") as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in user_values.items():
        writer.writerow([key, value])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

user_dict_2 = []

for user in data.id.unique():
    user_temp = user
    user = data[data['id'] == user]
    user_mood = user[user["variable"] == "mood"]

    avgs = []

    moods = list(user_mood["value"])

    size = int(len(moods) * 0.80)
    train, test = moods[0:size], moods[size:len(moods)]
    
    for q in [0, 1, 2, 3]:
        for p in [0, 1, 2, 3, 4, 5]:
            print(user_temp, p, 1, q)
            history = [x for x in train]
            predictions = list()
            try:
                for t in range(len(test)):
                    
                    model = ARIMA(history, order=(p, 1, q))
                    model_fit = model.fit(disp=0)
                    output = model_fit.forecast()
                    yhat = output[0]
                    predictions.append(yhat)
                    obs = test[t]
                    history.append(obs)
                    print('predicted=%f, expected=%f' % (yhat, obs))
                error = mean_squared_error(test, predictions)
                print('Test MSE: %.3f' % error)
                user_dict_2.append([user_temp, (p, 1, q), error])
            except:
                print("Didn't work for {0}, {1}, {2}".format(p, 0, q))
                pass

In [ ]:
user_values_2 = {}

for temp_inst in user_dict_2:
    if temp_inst[0] not in user_values_2:
        user_values_2[temp_inst[0]] = []
    user_values_2[temp_inst[0]].append([temp_inst[1], temp_inst[2]])

In [ ]:
# print(user_values_2)

with open('dict_1.csv', 'w', newline="") as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in user_values_2.items():
        writer.writerow([key, value])

In [5]:
import csv
with open('dict.csv', 'r') as temp_csv:
    csv_reader = csv.reader(temp_csv)

    items = []
    
    for row in csv_reader:
        last = 20.0
        for char in row[1].replace("[", "").replace("]","").split(", "):
            try:
                temp = float(char)
                if temp > 0.0 and temp < last:
                    last = temp
            except ValueError:
                pass
        items.append(last)
    
# print(items)

d_0 = []

with open('dict.csv', 'r') as temp_csv:
    csv_reader = csv.reader(temp_csv)
    j = 0
    for row in csv_reader:
        for i, char in enumerate(row[1].replace("[", "").replace("]","").split(", ")):
            try:
                if float(char) == items[j]:
#                     print(items[j], row[0], row[1].replace("[", "").replace("]","").split(", ")[i-3][1], row[1].replace("[", "").replace("]","").split(", ")[i-2][0], row[1].replace("[", "").replace("]","").split(", ")[i-1][0])
                    d_0.append([items[j], row[0], row[1].replace("[", "").replace("]","").split(", ")[i-3][1], row[1].replace("[", "").replace("]","").split(", ")[i-2][0], row[1].replace("[", "").replace("]","").split(", ")[i-1][0]])
            except ValueError:
                pass
        j += 1

# print(d_0)

with open('dict_1.csv', 'r') as temp_csv:
    csv_reader = csv.reader(temp_csv)

    items = []
    
    for row in csv_reader:
        last = 50.0
        for char in row[1].replace("[", "").replace("]","").split(", "):
            try:
                temp = float(char)
                if temp > 0.0 and temp != 1.0 and temp < last:
                    last = temp
            except ValueError:
                pass

        items.append(last)
    
# print(items)

d_1 = []

with open('dict_1.csv', 'r') as temp_csv:
    csv_reader = csv.reader(temp_csv)
    j = 0
    for row in csv_reader:
        for i, char in enumerate(row[1].replace("[", "").replace("]","").split(", ")):
#             print(char)
            try:
                if float(char) == items[j]:
                    d_1.append([items[j], row[0], row[1].replace("[", "").replace("]","").split(", ")[i-3][1], row[1].replace("[", "").replace("]","").split(", ")[i-2], row[1].replace("[", "").replace("]","").split(", ")[i-1][0]])
            except ValueError:
                pass
        j += 1
        
# print(d_1)

In [7]:
def run_arima_post_process(user_temp, parameters):
    p, d, q = parameters
    user = data[data['id'] == user_temp]
    user_mood = user[user["variable"] == "mood"]

    avgs = []

    times = list(user_mood["time"])
    moods = list(user_mood["value"])


    size = int(len(moods) * 0.8)
    train, test = moods[0:size], moods[size:len(moods)]
    time_test = times[size:len(times)]
    history = [x for x in train]
    predictions = list()

    date_avg_pred = {}
    accum_pred = 0
    temp_counter = 0
    counter = 0

    df_data =  []

    for t in range(len(test)):
        model = ARIMA(history, order=(1, 1, 1))
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        temp_t = time_test[t]
        history.append(obs)

        if int(temp_t[8:10]) != counter and t != 0:
            average = accum_pred / temp_counter
#             print("Day = {0}, Accumulated = {1}, Prediction in day = {2}, Average = {3}".format(time_test[t-1][0:10], accum_pred, temp_counter, average))
            df_data.append([user_temp, (p, d, q), time_test[t-1][0:10], average])
            date_avg_pred[counter] = average
            accum_pred = 0
            temp_counter = 0

    #     print("{}".format(temp_t[8:10]))
    #     print('%f' % (yhat))

        accum_pred += yhat[0]

        if t == len(test) - 1:
            average = accum_pred / (temp_counter + 1)
#             print("Day = {0}, Accumulated = {1}, Prediction in day = {2}, Average = {3}".format(time_test[t][0:10], accum_pred, temp_counter + 1, average))
            df_data.append([user_temp, (p, d, q), time_test[t][0:10], average])
            date_avg_pred[counter] = average

        counter = int(temp_t[8:10])
        temp_counter += 1

    error = mean_squared_error(test, predictions)
#     print('Test MSE: %.3f' % error)
    df_x = pd.DataFrame(df_data, columns=["id", "parameters", "date", "moodPred"])
    return df_x

In [13]:
if d_0[0][0] < d_1[0][0]:
    df_arima = run_arima_post_process(d_0[0][1], (d_0[0][2],d_0[0][3], d_0[0][4]))
    print(df_arima)
if d_1[0][0] < d_0[0][0]:
    df_arima = run_arima_post_process(d_1[0][1], (d_1[0][2],d_1[0][3], d_1[0][4]))
    print(df_arima)
for i in range(1, len(d_0)):
    if d_0[i][0] < d_1[i][0]:
        print("-----")
        df_temp = run_arima_post_process(d_0[i][1], (d_0[i][2],d_0[i][3], d_0[i][4]))
        print(df_temp)
        df_arima.append(df_temp)
    if d_1[i][0] < d_0[i][0]:
        print("-----")
        df_temp = run_arima_post_process(d_1[i][1], (d_1[i][2],d_1[i][3], d_1[i][4]))
        print(df_temp)
        df_arima.append(df_temp)
        
print(df_arima)
                                               


        id parameters        date  moodPred
0  AS14.01  (5, 1, 1)  2014-04-25  7.390926
1  AS14.01  (5, 1, 1)  2014-04-26  7.515843
2  AS14.01  (5, 1, 1)  2014-04-27  7.536188
3  AS14.01  (5, 1, 1)  2014-04-28  7.667148
4  AS14.01  (5, 1, 1)  2014-04-29  7.386527
5  AS14.01  (5, 1, 1)  2014-04-30  7.610674
6  AS14.01  (5, 1, 1)  2014-05-01  7.705187
7  AS14.01  (5, 1, 1)  2014-05-02  7.698041
8  AS14.01  (5, 1, 1)  2014-05-03  7.726928
9  AS14.01  (5, 1, 1)  2014-05-04  7.811843
-----
         id parameters        date  moodPred
0   AS14.02  (4, 1, 1)  2014-04-14  7.166701
1   AS14.02  (4, 1, 1)  2014-04-15  6.842102
2   AS14.02  (4, 1, 1)  2014-04-16  6.881859
3   AS14.02  (4, 1, 1)  2014-04-17  6.585240
4   AS14.02  (4, 1, 1)  2014-04-18  6.733991
5   AS14.02  (4, 1, 1)  2014-04-19  6.629337
6   AS14.02  (4, 1, 1)  2014-04-20  6.261690
7   AS14.02  (4, 1, 1)  2014-04-21  6.291868
8   AS14.02  (4, 1, 1)  2014-04-22  6.054598
9   AS14.02  (4, 1, 1)  2014-04-23  5.846621
10  AS14.02  (4

C:\Users\folke\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


         id parameters        date  moodPred
0   AS14.06  (4, 0, 2)  2014-04-27  7.043016
1   AS14.06  (4, 0, 2)  2014-04-28  6.964269
2   AS14.06  (4, 0, 2)  2014-04-29  7.020359
3   AS14.06  (4, 0, 2)  2014-04-30  7.035198
4   AS14.06  (4, 0, 2)  2014-05-01  7.122378
5   AS14.06  (4, 0, 2)  2014-05-02  7.077140
6   AS14.06  (4, 0, 2)  2014-05-03  6.978419
7   AS14.06  (4, 0, 2)  2014-05-04  7.091304
8   AS14.06  (4, 0, 2)  2014-05-05  7.093813
9   AS14.06  (4, 0, 2)  2014-05-07  7.041244
10  AS14.06  (4, 0, 2)  2014-05-08  7.032925
-----
         id parameters        date  moodPred
0   AS14.07  (2, 1, 1)  2014-04-24  5.561643
1   AS14.07  (2, 1, 1)  2014-04-25  5.426365
2   AS14.07  (2, 1, 1)  2014-04-26  5.320029
3   AS14.07  (2, 1, 1)  2014-04-27  5.285169
4   AS14.07  (2, 1, 1)  2014-04-28  5.316920
5   AS14.07  (2, 1, 1)  2014-04-29  5.365633
6   AS14.07  (2, 1, 1)  2014-04-30  5.387574
7   AS14.07  (2, 1, 1)  2014-05-01  5.338160
8   AS14.07  (2, 1, 1)  2014-05-02  5.278365
9   

C:\Users\folke\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


         id parameters        date  moodPred
0   AS14.15  (3, 0, 0)  2014-04-26  6.672524
1   AS14.15  (3, 0, 0)  2014-04-27  6.809208
2   AS14.15  (3, 0, 0)  2014-04-28  6.919928
3   AS14.15  (3, 0, 0)  2014-04-29  6.943250
4   AS14.15  (3, 0, 0)  2014-04-30  6.598867
5   AS14.15  (3, 0, 0)  2014-05-01  6.545847
6   AS14.15  (3, 0, 0)  2014-05-02  6.834980
7   AS14.15  (3, 0, 0)  2014-05-03  6.770374
8   AS14.15  (3, 0, 0)  2014-05-04  6.879652
9   AS14.15  (3, 0, 0)  2014-05-05  6.904630
10  AS14.15  (3, 0, 0)  2014-05-07  6.925259
11  AS14.15  (3, 0, 0)  2014-05-08  6.939237
-----
         id parameters        date  moodPred
0   AS14.16  (0, 1, 1)  2014-04-22  6.898890
1   AS14.16  (0, 1, 1)  2014-04-23  6.927661
2   AS14.16  (0, 1, 1)  2014-04-24  6.659217
3   AS14.16  (0, 1, 1)  2014-04-25  6.581334
4   AS14.16  (0, 1, 1)  2014-04-26  6.679489
5   AS14.16  (0, 1, 1)  2014-04-27  6.773601
6   AS14.16  (0, 1, 1)  2014-04-28  6.947451
7   AS14.16  (0, 1, 1)  2014-04-29  7.024250
8   

...

        id parameters        date  moodPred
0  AS14.30  (1, 1, 3)  2014-04-26  7.810476
1  AS14.30  (1, 1, 3)  2014-04-27  7.817436
2  AS14.30  (1, 1, 3)  2014-04-28  7.846534
3  AS14.30  (1, 1, 3)  2014-04-29  7.866029
4  AS14.30  (1, 1, 3)  2014-04-30  7.883716
5  AS14.30  (1, 1, 3)  2014-05-01  7.888574
6  AS14.30  (1, 1, 3)  2014-05-02  7.914628
7  AS14.30  (1, 1, 3)  2014-05-03  7.905317
8  AS14.30  (1, 1, 3)  2014-05-04  7.897901
9  AS14.30  (1, 1, 3)  2014-05-05  7.916613
-----
        id parameters        date  moodPred
0  AS14.31  (0, 0, 2)  2014-05-07  7.064609
1  AS14.31  (0, 0, 2)  2014-05-08  7.083982
2  AS14.31  (0, 0, 2)  2014-05-09  7.080425
3  AS14.31  (0, 0, 2)  2014-05-10  7.109813
4  AS14.31  (0, 0, 2)  2014-05-11  7.141158
5  AS14.31  (0, 0, 2)  2014-05-12  7.103800
6  AS14.31  (0, 0, 2)  2014-05-13  7.097992
7  AS14.31  (0, 0, 2)  2014-05-14  7.092682
8  AS14.31  (0, 0, 2)  2014-05-15  7.089174
-----


C:\Users\folke\Anaconda3\lib\site-packages\statsmodels\base\model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\folke\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


        id parameters        date  moodPred
0  AS14.32  (4, 1, 2)  2014-05-07  7.069723
1  AS14.32  (4, 1, 2)  2014-05-08  6.875437
2  AS14.32  (4, 1, 2)  2014-05-09  7.014376
3  AS14.32  (4, 1, 2)  2014-05-10  7.223753
4  AS14.32  (4, 1, 2)  2014-05-11  7.118330
5  AS14.32  (4, 1, 2)  2014-05-12  7.100806
-----


C:\Users\folke\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


        id parameters        date  moodPred
0  AS14.33  (5, 0, 1)  2014-05-23  5.873520
1  AS14.33  (5, 0, 1)  2014-05-24  5.930951
2  AS14.33  (5, 0, 1)  2014-05-25  5.983302
3  AS14.33  (5, 0, 1)  2014-05-26  5.834044
4  AS14.33  (5, 0, 1)  2014-05-27  5.836650
5  AS14.33  (5, 0, 1)  2014-05-28  6.354672
6  AS14.33  (5, 0, 1)  2014-05-29  6.173488
7  AS14.33  (5, 0, 1)  2014-05-30  6.280235
8  AS14.33  (5, 0, 1)  2014-05-31  6.108167
        id parameters        date  moodPred
0  AS14.01  (5, 1, 1)  2014-04-25  7.390926
1  AS14.01  (5, 1, 1)  2014-04-26  7.515843
2  AS14.01  (5, 1, 1)  2014-04-27  7.536188
3  AS14.01  (5, 1, 1)  2014-04-28  7.667148
4  AS14.01  (5, 1, 1)  2014-04-29  7.386527
5  AS14.01  (5, 1, 1)  2014-04-30  7.610674
6  AS14.01  (5, 1, 1)  2014-05-01  7.705187
7  AS14.01  (5, 1, 1)  2014-05-02  7.698041
8  AS14.01  (5, 1, 1)  2014-05-03  7.726928
9  AS14.01  (5, 1, 1)  2014-05-04  7.811843


AS14.01 0.328 (10 0 2)
AS14.05 0.382 (8 0 0)
AS14.13 0.694 (7 0 1)

In [ ]:
lag_acf = acf(moods, nlags=300)
#Plot PACF:
plt.figure(figsize=(16, 7))
plt.plot(lag_acf,marker='+')
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(moods)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(moods)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')
plt.xlabel('number of lags')
plt.ylabel('correlation')
plt.tight_layout()

#calling partial correlation function
lag_pacf = pacf(moods, nlags=30, method='ols')
#Plot PACF:
plt.figure(figsize=(16, 7))
plt.plot(lag_pacf,marker='+')
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(moods)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(moods)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.xlabel('Number of lags')
plt.ylabel('correlation')
plt.tight_layout()

In [ ]:
model = ARIMA(moods, order=(1, 1, 1))
model_fit = model.fit(disp=0)
print(model_fit.summary())
residuals = DataFrame(model_fit.resid)
residuals.plot()
plt.show()
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())

In [ ]:
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA

size = int(len(moods) * 0.66)

train, test = moods[0:size], moods[size:len(moods)]
history = [x for x in train]
predictions = list()

for t in range(len(test)):
    model = ARIMA(history, order=(12, 0, 2))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()